### 1. Prise en mains de igraph :

In [50]:
!pip install -q graphframes

In [46]:
import igraph as ig
import os
# # !find /usr/local -name "pyspark"
os.environ["SPARK_HOME"] = "/home/djeghali/spark"
os.environ["JAVA_HOME"] = "/usr"


In [47]:
# Main imports
import findspark
from pyspark.sql import SparkSession
from pyspark import SparkConf

# for dataframe and udf
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *

import pandas as pd


# initialise environment variables for spark
findspark.init()

# Start spark session
# --------------------------
def start_spark():
  local = "local[*]"
  appName = "TP"

  gf = "graphframes:graphframes:0.8.3-spark3.5-s_2.12"

  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "6G").\
  set("spark.driver.memory","6G").\
  set("spark.sql.catalogImplementation","in-memory").\
  set("spark.jars.packages", gf)

  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")

  spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")

  # Adjust the query execution environment to the size of the cluster (4 cores)
  spark.conf.set("spark.sql.shuffle.partitions","4")
  print("session started, its id is ", sc.applicationId)
  return spark
spark = start_spark()


session started, its id is  local-1707047423644


In [80]:

#Import GraphFrames
from graphframes import GraphFrame
from graphframes.lib import AggregateMessages as AM
# from graphframes.lib import Pregel

#For connectedComponents()
# !pwd
# !mkdir -p /content/checkpoints
# spark.sparkContext.setCheckpointDir('/content/checkpoints')

#Import networkx
import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx


# Initialisez une session Spark
spark = SparkSession.builder.appName("LectureParquet").getOrCreate()
# Chemin vers le fichier Parquet compressé avec Snappy
parquet_file_10K = "local/data/BDLE_10K/*.snappy.parquet"
parquet_file_1M = "local/data/BDLE_1M/*.snappy.parquet"

# Lisez le fichier Parquet dans un DataFrame PySpark
data_10K = spark.read.format("parquet").option("compression", "snappy").load(parquet_file_10K)
data_1M = spark.read.format("parquet").option("compression", "snappy").load(parquet_file_1M)





In [81]:
# Affichez le contenu du DataFrame
data_10K.show()
print("Nombre de tuples dans les données de taille 10K  : ",data_10K.count())

+-----------+-----------+-----+
|     seqID1|     seqID2|  sim|
+-----------+-----------+-----+
|117761605:5|152890023:5| 97.8|
|152904885:3|155591878:2| 88.5|
|152887848:4|153682181:0|100.0|
|152937692:5| 80009514:2| 82.3|
|152990923:2|154549183:4| 98.0|
|152867782:1|153171917:1| 83.1|
| 15111981:2|153137370:1|100.0|
|152794195:0| 15280704:2| 96.9|
| 62963742:1| 63783418:5| 98.4|
|152170568:3|153062631:2| 97.6|
|152904832:4|154500443:5| 98.6|
|152745429:3| 22988511:2|100.0|
|153161980:3|154256473:0|100.0|
|152903373:4|154426964:5| 80.4|
|146010871:1|158434400:4| 98.7|
|153040013:0|156563807:1| 96.5|
|152933621:4|157634534:2| 96.8|
|153141720:0|155282497:0| 93.2|
|153082115:5|153887336:3| 90.0|
|153025447:3|155660759:1| 97.4|
+-----------+-----------+-----+
only showing top 20 rows

Nombre de tuples dans les données de taille 10K  :  20842


In [82]:
# Affichez le contenu du DataFrame 1M
data_1M.show()
print("Nombre de tuples dans les données de taille 1M : ",data_1M.count())

+--------------------+--------------------+-----+
|              seqID1|              seqID2|  sim|
+--------------------+--------------------+-----+
|          28266027:2|          30697475:4|100.0|
|         141099253:1|         142374057:2| 87.8|
|         100479579:0|          23509310:0| 83.9|
|         113579361:3|         114583521:5|100.0|
|         127898390:3|         142186630:2|100.0|
|          19430332:0|          78781103:4| 85.4|
|         141834541:2|         145255331:5|100.0|
|         103259805:4|          58392840:4| 83.3|
|         122611397:5|         143667719:3|100.0|
|          23535857:4|           4695744:1| 95.8|
|          41064011:0|          58827273:4|100.0|
|         107823489:0|         107855124:4| 85.1|
|METdb_00146-1-DN1...|METdb_00154-1-DN1...| 92.6|
|         156037355:1|         157677656:3| 88.9|
|          36873489:1|          89199423:1| 93.1|
|         119223634:3|         119895769:4| 88.7|
|         126499401:5|         140644518:1|100.0|


In [83]:
# On converti en dataframe pandas :
data_10K = data_10K.toPandas()
data_1M = data_1M.toPandas()

-   Génération des igraphes : 

In [84]:
# Créarion d'un graphe  à partir des données lues :
data_10K_ig_graph = ig.Graph.TupleList(data_10K[['seqID1', 'seqID2']].itertuples(index=False), directed=False)
data_1M_ig_graph = ig.Graph.TupleList(data_1M[['seqID1', 'seqID2']].itertuples(index=False), directed=False)


-   Calcule du temps de calcule des composantes connexes : 

    -   Data 10 K : 

In [85]:
from time import time

start_time = time()

# Calculer les composantes connexes
components = data_10K_ig_graph.components()

# Afficher les composantes connexes
# print("Composantes connexes :", components)

# Afficher le temps d'exécution
end_time = time()
execution_time_10K = end_time - start_time

print("Temps d'exécution :", execution_time_10K, "secondes")


Temps d'exécution : 0.01446843147277832 secondes


    -   Data 1 M : 

In [86]:
from time import time

start_time = time()

# Calculer les composantes connexes
components = data_1M_ig_graph.components()

# Afficher les composantes connexes
# print("Composantes connexes :", components)

# Afficher le temps d'exécution
end_time = time()
execution_time_1M = end_time - start_time

print("Temps d'exécution :", execution_time_1M, "secondes")


Temps d'exécution : 0.5444376468658447 secondes


### Vérifier le multiprocessing de igraphe :


In [100]:
import multiprocessing
from igraph import Graph

def process_1():
    # Code du premier programme
    components = data_1M_ig_graph.components()

def process_2():
    # Code du deuxième programme
    components = data_1M_ig_graph.components()

if __name__ == "__main__":

    # Calculer les composantes connexes
    # Créer deux processus pour exécuter les deux programmes en parallèle
    p1 = multiprocessing.Process(target=process_1)
    p2 = multiprocessing.Process(target=process_2)

    start_time = time()

    # Démarrer les deux processus
    p1.start()
    p2.start()

    # Attendre que les deux processus se terminent
    p1.join()
    p2.join()

    # Afficher le temps d'exécution
    end_time = time()
    execution_time_2proccessus = end_time - start_time

    print("Temps d'exécution paralèlle :", execution_time_2proccessus, "secondes")


Temps d'exécution paralèlle : 0.9021463394165039 secondes


In [106]:
import time
from multiprocessing import Pool

def process_subgraph(subgraph):
    # Effectuez ici le traitement sur le sous-graphe
    # Par exemple, calculer une métrique spécifique
    result = subgraph.components()
    return result

def main():
    # Chargez votre graphe à partir de votre source de données
    # graph = ...

    # Divisez le graphe en sous-graphes (vous pouvez ajuster la logique en fonction de votre besoin)
    subgraphs = [data_1M_ig_graph,data_1M_ig_graph]

    start_time = time.time()

    # Utilisez multiprocessing pour paralléliser le traitement des sous-graphes
    with Pool() as pool:
        results = pool.map(process_subgraph, subgraphs)

    end_time = time.time()


    total_time = end_time - start_time
    print(f"Temps total de traitement : {total_time} secondes")

if __name__ == "__main__":
    main()


Temps total de traitement : 13.76425576210022 secondes
